### --- Step 1: Setup and Imports ---


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, f1_score, confusion_matrix
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
import hopsworks
import exclude.key

# Hopsworks
HOPSWORKS_API_KEY = exclude.key.HOPSWORKS_API_KEY
FEATURE_GROUP_NAME = "bars_near_london_bridge"
FEATURE_GROUP_VERSION = 1

# Connect to Hopsworks
project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/25749
Connected. Call `.close()` to terminate connection gracefully.


### --- Step 2: Load Data from the Feature group ---


In [14]:
# Retrieve feature groups.
fg = fs.get_feature_group(
    name=FEATURE_GROUP_NAME, 
    version=FEATURE_GROUP_VERSION,
)

# Select features for training datasets
df = fg.read()
df.head(5)

2024-05-31 17:47:49,255 WARNING: DeprecationWarning: DatetimeTZBlock is deprecated and will be removed in a future version. Use public APIs instead.

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.06s) 


,venue_name,venue_address,day,hour,busyness,last_updated
0,All Bar One Tower Of London,"14 Byward St, Greater, London EC3R 5BA, United...",Monday,19,Average,2024-05-31 09:05:59.051218+00:00
1,The Old School Yard,"111 Long Ln, London SE1 4PH, United Kingdom",Tuesday,19,Low,2024-05-31 09:05:59.051218+00:00
2,The Two Bridges Ale House & Kitchen,"186 Tooley St, London SE1 2TZ, United Kingdom",Saturday,0,Average,2024-05-31 09:05:59.051218+00:00
3,The Kings Arms,"251 Tooley St, London SE1 2JX, United Kingdom",Sunday,3,Closed,2024-05-31 09:05:59.051218+00:00
4,The Market Porter,"9 Stoney St, London SE1 9AA, United Kingdom",Sunday,2,Closed,2024-05-31 09:05:59.051218+00:00


### --- Step 3: Process the Data ---

In [15]:
# # Map categorical values of 'busyness' to numeric values
# busyness_mapping = {'Closed': 0, 'Low': 1, 'Below average':2, 'Average': 3, 'Above average':4, 'High': 5}
# df['busyness'] = df['busyness'].map(busyness_mapping)

# Load transformation functions from the feature store
min_max_scaler = fs.get_transformation_function(name="min_max_scaler")
label_encoder = fs.get_transformation_function(name="label_encoder")

# Define numerical and categorical features
numerical_features = ["hour"]
categorical_features = ["venue_name", "venue_address", "day", "busyness"]

# Map features to transformation functions
transformation_functions = {}
for feature in numerical_features:
    transformation_functions[feature] = min_max_scaler
for feature in categorical_features:
    transformation_functions[feature] = label_encoder

df.head(5)

,venue_name,venue_address,day,hour,busyness,last_updated
0,All Bar One Tower Of London,"14 Byward St, Greater, London EC3R 5BA, United...",Monday,19,Average,2024-05-31 09:05:59.051218+00:00
1,The Old School Yard,"111 Long Ln, London SE1 4PH, United Kingdom",Tuesday,19,Low,2024-05-31 09:05:59.051218+00:00
2,The Two Bridges Ale House & Kitchen,"186 Tooley St, London SE1 2TZ, United Kingdom",Saturday,0,Average,2024-05-31 09:05:59.051218+00:00
3,The Kings Arms,"251 Tooley St, London SE1 2JX, United Kingdom",Sunday,3,Closed,2024-05-31 09:05:59.051218+00:00
4,The Market Porter,"9 Stoney St, London SE1 9AA, United Kingdom",Sunday,2,Closed,2024-05-31 09:05:59.051218+00:00


### --- Step 4: Create the feature view ---

In [16]:
feature_view = fs.get_or_create_feature_view(
    name='busyness_feature_view',
    version=2,
    query=fg.select_all(),
    labels=["busyness"],
    transformation_functions=transformation_functions,
)

# if the featureview is already created just retrieve it 
# feature_view = fs.get_feature_view(name='busyness_feature_view', version=1)


ValueError: Online transformations for training dataset labels are not supported.

### --- Step 5: Create the training dataset ---

In [7]:
# Split data into training, validation, and test sets
X_train, X_val, X_test, y_train, y_val, y_test = feature_view.train_validation_test_split(
    validation_size=0.2,
    test_size=0.1,
)

# Drop unnecessary columns
X_train.drop(['venue_address', 'last_updated'], axis=1, inplace=True)
X_val.drop(['venue_address', 'last_updated'], axis=1, inplace=True)
X_test.drop(['venue_address', 'last_updated'], axis=1, inplace=True)

# # Add the popularity_score feature to the training, validation, and test sets
# X_train['popularity_score'] = X_train['busyness'].rolling(window=3, min_periods=1).mean()
# X_val['popularity_score'] = X_val['busyness'].rolling(window=3, min_periods=1).mean()
# X_test['popularity_score'] = X_test['busyness'].rolling(window=3, min_periods=1).mean()

X_train, X_val, X_test, y_train, y_val, y_test.head(3)

2024-05-31 17:42:43,635 WARNING: DeprecationWarning: DatetimeTZBlock is deprecated and will be removed in a future version. Use public APIs instead.

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.64s) 
2024-05-31 17:42:46,633 WARNING: VersionWarning: Incremented version to `9`.



(      venue_name  day      hour
 0              0    0  0.826087
 1              1    1  0.826087
 3              2    2  0.130435
 4              3    2  0.086957
 5              4    3  0.260870
 ...          ...  ...       ...
 4361           9    1  0.347826
 4362           1    0  0.347826
 4364           5    4  0.782609
 4366          11    0  0.434783
 4367          17    3  0.521739
 
 [3057 rows x 3 columns],
       venue_name  day      hour
 2             19    4  0.000000
 9             25    2  0.304348
 10             9    5  0.434783
 12            22    6  0.130435
 13            22    4  0.869565
 ...          ...  ...       ...
 4351          20    0  0.434783
 4355          12    2  0.347826
 4356           2    4  0.565217
 4363          11    6  0.391304
 4365           9    0  0.434783
 
 [873 rows x 3 columns],
       venue_name  day      hour
 18             8    6  0.695652
 20            10    0  1.000000
 25            20    1  0.913043
 35             3    

### --- Step 6: Train amd evaluate the model ---

In [6]:

# Create an instance of the XGBClassifier 
model = xgb.XGBClassifier()
model.fit(X_train, y_train)

# Predict and evaluate
y_pred_train = model.predict(X_train)
y_pred_val = model.predict(X_val)
y_pred_test = model.predict(X_test)

# Compute metrics
metrics = {"f1_score": f1_score(y_test, y_pred_test, average='macro')}
print(metrics)

# Confusion matrix
results = confusion_matrix(y_test, y_pred_test, labels=[0, 1])
df_cm = pd.DataFrame(results, ['True Low', 'True High'], ['Pred Low', 'Pred High'])
cm = sns.heatmap(df_cm, annot=True)
cm.get_figure().show()


ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2 3 4 5], got ['Above average' 'Average' 'Below average' 'Closed' 'High' 'Low']